In [1]:
pip install -q tensorflow-model-optimization

     |████████████████████████████████| 174kB 3.7MB/s 


In [2]:
from keras.models import load_model
model=load_model('global_model_100.h5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                48        
_________________________________________________________________
dense_2 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_3 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 6,929
Trainable params: 6,929
Non-trainable params: 0
_________________________________________________________________


In [3]:
import tensorflow_model_optimization as tfmot
from tensorflow import keras

quantize_model = tfmot.quantization.keras.quantize_model
# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)
optimizer = keras.optimizers.Adam(lr=0.0001)
# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer=optimizer,loss='mean_squared_error')
q_aware_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer (QuantizeLaye (None, 2)                 3         
_________________________________________________________________
quant_dense_1 (QuantizeWrapp (None, 16)                53        
_________________________________________________________________
quant_dense_2 (QuantizeWrapp (None, 32)                549       
_________________________________________________________________
quant_dense_3 (QuantizeWrapp (None, 64)                2117      
_________________________________________________________________
quant_dense_4 (QuantizeWrapp (None, 64)                4165      
_________________________________________________________________
quant_dense_5 (QuantizeWrapp (None, 1)                 70        
Total params: 6,957
Trainable params: 6,929
Non-trainable params: 28
___________________________________________________

In [4]:
q_aware_model.save('Intel_quantize_aware_model.h5')

In [8]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]

optimize="Speed"
if optimize=='Speed':
    converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_LATENCY]
elif optimize=='Storage':
     converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
else:    
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
#reduce the size of a floating point model by quantizing the weights to float16
converter.target_spec.supported_types = [tf.float16]
#save the quanitized model toa binary file

quantized_tflite_model = converter.convert()
with open('Intel_QAT.tflite16', 'wb') as f:
  f.write(quantized_tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp3yf67vb6/assets


INFO:tensorflow:Assets written to: /tmp/tmp3yf67vb6/assets


In [32]:
# TensorFlow Lite converts full floating point to 8-bit integers
# import tensorflow as tf
# converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# quantized_tflite_model = converter.convert()
# with open('Intel_QAT.tflite', 'wb') as f:
#   f.write(quantized_tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp1c9iewrn/assets


INFO:tensorflow:Assets written to: /tmp/tmp1c9iewrn/assets
